# Using GenePattern Files in Python

This is a short tutorial on how to use GenePattern files in conjunction with common Python libraries, such as [csv](https://docs.python.org/2/library/csv.html), [numpy](http://www.numpy.org/), [pandas](http://pandas.pydata.org/) and [matplotlib](http://matplotlib.org/). It demonstrates how to load data from GenePattern into these libraries, and how to send data from these libraries back to GenePattern.

This tutorial assumes that the reader is familiar with [GenePattern](http://genepattern.org) and its basic associated concepts, such as modules, jobs and result files. A simple GenePattern tutorial is [available here](http://www.broadinstitute.org/cancer/software/genepattern/tutorial). It also assumes that the reader is familiar with the basics of the [GenePattern Python library](http://www.broadinstitute.org/cancer/software/genepattern/programmers-guide#_Using_GenePattern_from_Python). If you are not already familiar with this library, it is recommended that you first complete the [GenePattern Python Tutorial](http://www.broadinstitute.org/cancer/software/genepattern/example-genepattern-notebooks).

### Installation

Several of the libraries in this tutorial must first be installed in order to use their functionality. Installation is typically done through PIP. All of the relevant libraries can be installed by executing the line below on the command line, with the necessary permissions. You may need to restart the IPython kernel before the libraries become available after installation.

> *pip install numpy pandas matplotlib genepattern-python*

## GPFile Objects

Files in the GenePattern Python library are represented by GPFile objects. These objects contain an URL to the file on the GenePattern server and several methods for obtaining the contents of the file.

GPFile objects can either be created directly or obtained from a GenePattern job, through the GPJob object. An example of directly instantiating a GPFile object to represent a remote file is below. 

An example of obtaining a GPFile object representing a GenePattern job's output is also given, but is commented out, as this tutorial does not cover launching GenePattern jobs.

In [ ]:
import gp

# Create a GenePattern server proxy instance
gpserver = gp.GPServer('https://genepattern.broadinstitute.org/gp','myusername', 'mypassword')

# Directly create the GPFile object
gpfile = gp.GPFile(gpserver, 
                   'ftp://ftp.broadinstitute.org/pub/genepattern/datasets/protocols/all_aml_test.preprocessed.gct')

# The code below demonstrates how to obtain a GPFile from a GPJob object.
# It asumes the user already has a GPJob launched and represented in the gpjob variable.

# This returns a list of all output files as GPFiles.
# output_list = gpjob.get_output_files()  

# The list can be iterated over or a single member obtained
# gpfile = output_list[0]

### Understanding Your File

When doing any sort of file parsing in Python, it is important to understand the contents of the file that you are using. The file we will use with this tutorial is a GCT containing expression data for different leukemia samples.

[Here is a link to a GCT file](http://www.broadinstitute.org/cancer/software/genepattern/tutorial/linkedFiles/allaml.dataset.gct). Go ahead and click the link. It will open up in your browser.

As you can see, the file consists of a couple lines with header information, followed by a tab-separated matrix of expression data. This will be important to keep in mind later in the tutorial, when we start loading the data using different libraries.

More information on the GCT format is [available here](http://www.broadinstitute.org/cancer/software/genepattern/file-formats-guide#GCT).

## Reading the File

Every GPFile object has a read() method, which downloads the file and returns its contents as a string. As strings are handled differently, depending on whether you are using Python 2 or Python 3, we recommend explicitly reading the contents as unicode. This will ensure that the file can be easily used in either version of Python.

An example of reading the contents of the file to a variable is given below.

In [ ]:
# This reads the file and casts its contents as unicode-8
raw_txt = unicode(gpfile.read(), "utf-8")

## GenePattern File IO

You now have the contents of the file represented as a string. With this you can read the data, parse the file or, with the right knowhow, perform analyses.

Many common libraries, however, expect data to come in the form of a Python IO object, rather than a raw string. Thankfully, the data can be used in this way simply by instantiating a StringIO object. The code below shows how to do this.

In [ ]:
from io import StringIO

# Wrap the data in a StringIO object
data_io = StringIO(raw_txt)

**Note:** The GPFile object also possesses the open() method. This method directly returns an IO object for the remote file, bypassing the need to first read the file and then wrap its contents with a StringIO. Unfortunately, due to the way that Python handles strings, this results in different behavior between Python 2 and Python 3. If you use this method and run into errors, we recommend trying the above method instead.

## Using GenePattern with the csv module

One of the simplest ways to parse the data is using Python's built-in [csv](https://docs.python.org/2/library/csv.html) module. This module provides a variety of methods for parsing text into matrices of raw values.

The most important method in this library is the reader() method. It will parse each line into a list of values and then return an object from which each line can be read in order. This can either be iterated over or cast as a list.

An example of this is shown below.

In [ ]:
import csv

# Load the data into a reader object.
# For a GCT file, the delimiter between columns is the tab character
csv_reader = csv.reader(data_io, delimiter='\t')

# Cast the reader object as a list of lists
data_matrix = list(csv_reader)

# Print the fourth line of the file (the first sample)
# Remember, the first line will be index 0.
print(data_matrix[3])

Note that the first two columns in this matrix will be the sample name and description. These correspond to the first two entries in the list printed above.

Also note that the csv library parses all values as strings, including of the numerical values in the list. Before analyses can be performed with these values, they will need to be parsed as numbers. This is a fundamental limitation of the csv library, and one other libraries avoid.

## Using GenePattern with numpy

The popular [numpy](http://www.numpy.org/) package provides a variety of tools useful for scientific computing. Among these tools are a method to quickly parse files into matrices of data. This method is named [genfromtxt()](http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.genfromtxt.html).

Below is an example of how to load the GCT file into a numpy matrix, using the IO object we created earlier. This method call requires that a number of parameters are set. This tailors the parsing of the data to fit the GCT format.

In [ ]:
import numpy

# Reset the state of the IO object.
# This prepares it for reading again.
data_io = StringIO(raw_txt)

# This reads the file and generates a matrix of values.
# The parameters are as follows:
#
#     fname:       This is the IO object
#     delimiter:   GCT files are tab-seperated
#     skip_header: Skip the first three rows with header info
#     dtype:       Determine the type of each column based on its contents
#     comments:    GCT files contain no comment lines

data_matrix = numpy.genfromtxt(fname=data_io, delimiter='\t', skip_header=3, dtype=None, comments=None)

# Print the first line of the matrix to demonstrate
print(data_matrix[0])

**Note:** You will notice that this method of parsing is able to recognize when a column's value is either a string or a number. This allows you to perform analyses on the data without first having to convert each numerical value to the appropriate type.

## Using GenePattern with pandas

[pandas](http://pandas.pydata.org/) is a popular data analysis library that provides a variety of tools and data structures for use in scientific computing. Additionally, pandas comes with a number of Jupyter integrations, making it easy to use from within a Jupyter notebook.

Among the tools provided by pandas is the *read_csv()* method. This method can be used to easily read a GenePattern file, and displays the results as a table. Example code demonstrating this is shown below.

In [ ]:
import pandas

# Reset the state of the IO object.
# This prepares it for reading again.
data_io = StringIO(raw_txt)

# This reads the file and generates data structure representing the values.
# If a code cell returns this value, it will be displayed in the notebook as a table.
# The parameters are as follows:
#
#     delimiter:   GCT files are tab-separated
#     header:      This line contains the column headers

pandas.read_csv(data_io, delimiter='\t', header=2)

**Note:** The pandas library requires certain versions of the numpy library in order to run. If you are experiencing errors when attempting to import pandas, you may have an incompatible version of numpy installed. This is particularly common on Mac OSX, as many Macs come with an older version of the numpy library. To fix this problem you will first need to remove the older version of the library from your Python path, and then install the newer version through PIP.

## Using GenePattern with matplotlib

[matplotlib](http://matplotlib.org/) is a popular plotting and graphing library. It comes with a variety of Jupyter integrations. These features are ideal for visualizing the contents of GenePattern files in ways that supplement GenePattern's native visualizers.

By default matplotlib displays its visualizations a separate Python window. This may or may not work for you, depending on whether you are running Jupyter locally. When working with matplotlib, it is good to tell the notebook to display all matplotlib visualizations inline. The magic for this is shown below.

In [ ]:
%matplotlib inline

The matplotlib library comes with far more display options than can be covered in this tutorial. Furthermore, doing so would be outside its scope. Instead, below we present a piece of code that demonstrates how to use pandas and matplotlib to display the GenePattern file as a heatmap. Note that for ease of demonstration and readability this code displays only a small portion of the GenePattern file.

In [ ]:
import matplotlib.cbook as cbook
import matplotlib.pyplot as plt
import pylab

# Reset the state of the IO object.
# This prepares it for reading again.
data_io = StringIO(raw_txt)

# Parse the file using pandas's read_csv() method, 
# as was used in the previous section
table = pandas.read_csv(data_io, delimiter='\t', header=2)

# Tell matplotlib to generate a 15 in x 10 in image
pylab.rcParams['figure.figsize'] = 15, 10

# For ease of demonstration, display only a portion of the GenePattern file
submatrix = table.ix[0:10,2:10].values

# Note that if we were doing real science in this demonstration, 
# we would normalize the values in the matrix somewhere around this point

# Obtain references to the figure and axis parts of the heatmap
fig, ax = plt.subplots()

# Set the color scheme used by the heatmap
heatmap = ax.pcolor(submatrix, cmap=plt.cm.bwr)

# Put the major ticks at the middle of each cell
ax.set_xticks(numpy.arange(submatrix.shape[1])+0.5, minor=False)
ax.set_yticks(numpy.arange(submatrix.shape[0])+0.5, minor=False)

# Use a more natural, table-like display
ax.invert_yaxis()
ax.xaxis.tick_top()

# Extract the column and row labels from the file
column_labels = [item[0] for item in table[[0]].values[0:10]]
row_labels = list(table.columns[2:13])

# Set the table and row labels for display
ax.set_xticklabels(column_labels, minor=False)
ax.set_yticklabels(row_labels, minor=False)

# Display the heatmap
plt.show()